In [6]:
import pickle
import numpy as np
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
from keras.callbacks import Callback
from sklearn.preprocessing import LabelEncoder

class F1ScoreCallback(Callback):
    def __init__(self, X_val, y_val):
        super(F1ScoreCallback, self).__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0
        self.best_model = None
        self.f1_scores = []

    def on_epoch_end(self, epoch, logs=None):
        y_val_pred = np.argmax(self.model.predict(self.X_val), axis=1)
        f1 = f1_score(self.y_val, y_val_pred, average='weighted')
        self.f1_scores.append(f1)
        

        if f1 > self.best_f1:
            self.best_f1 = f1
            self.best_model = self.model
            print(f"Epoch {epoch + 1} - F1 Score: {f1:.4f}")
            print("Saved best model")
            print(self.f1_scores)

with open('train_without_test4.pickle', 'rb') as f1:
    balanced = pickle.load(f1)

with open('test_descr_no_subj_with_embeddings.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

train = np.array([item['cve_description_no_subject_ada_embedding'] for item in balanced if item['cwe'] != 'None'])
test = np.array([item['cwe'] for item in balanced if item['cwe'] != 'None'])
np.random.seed(42)
X_train, X_val, y_train, y_val = train_test_split(train,test,test_size=0.1,random_state=42)

X_test = np.array([item['cve_description_no_subject_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

label_encoder_train = LabelEncoder()
y_train_encoded = label_encoder_train.fit_transform(y_train)
label_encoder_test = LabelEncoder()
y_test_encoded = label_encoder_test.fit_transform(y_test)


input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train))

model = Sequential()
model.add(Dense(128, input_dim=input_dim, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

f1_callback = F1ScoreCallback(X_val, label_encoder_train.transform(y_val))

history = model.fit(X_train, y_train_encoded, epochs=60, batch_size=32, validation_data=(X_val, label_encoder_train.transform(y_val)), verbose=1, callbacks=[f1_callback])

best_model = f1_callback.best_model


# Save the best model
joblib.dump(best_model, 'CWE_classes.joblib')

# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_pred_original = label_encoder_train.inverse_transform(y_pred)

print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))

joblib.dump(label_encoder_train, 'label_encoder_train.joblib')

Epoch 1/60
260/260 [==============================] - 1s 1ms/step loss: 2.1129 - a
Epoch 1 - F1 Score: 0.5187
Saved best model
[0.518681627990273]
2336/2336 [==============================] - 11s 4ms/step - loss: 2.1129 - accuracy: 0.4432 - val_loss: 1.6214 - val_accuracy: 0.5508
Epoch 2/60
260/260 [==============================] - 1s 2ms/step loss: 1.52
Epoch 2 - F1 Score: 0.5553
Saved best model
[0.518681627990273, 0.5552916137714813]
2336/2336 [==============================] - 10s 4ms/step - loss: 1.5282 - accuracy: 0.5729 - val_loss: 1.4813 - val_accuracy: 0.5829
Epoch 3/60
260/260 [==============================] - 1s 2ms/step loss: 1.405
Epoch 3 - F1 Score: 0.5870
Saved best model
[0.518681627990273, 0.5552916137714813, 0.5870459488251618]
2336/2336 [==============================] - 10s 4ms/step - loss: 1.4050 - accuracy: 0.6035 - val_loss: 1.4082 - val_accuracy: 0.6073
Epoch 4/60
260/260 [==============================] - 1s 2ms/step loss: 1.3344
Epoch 4 - F1 Score: 0.6051
Sa

['label_encoder_train.joblib']

Run inference 

In [3]:
import pickle
import numpy as np
from sklearn.metrics import classification_report
import joblib
from sklearn.preprocessing import LabelEncoder
# Save the best model
with open('test_descr_no_subj_with_embeddings.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

X_test = np.array([item['cve_description_no_subject_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

best_model=joblib.load('best_model.joblib')
label_encoder_train=joblib.load('label_encoder_train.joblib')
# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_pred_original = label_encoder_train.inverse_transform(y_pred)

print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))

444/444 [==============================] - 1s 2ms/step
Classification Report:
               precision    recall  f1-score   support

         119     0.7319    0.3598    0.4825      1070
         120     0.3333    0.5969    0.4278       196
         125     0.7591    0.7876    0.7731       532
         134     0.3030    0.5263    0.3846        19
         189     0.3958    0.3193    0.3535       119
         190     0.5565    0.6900    0.6161       200
          20     0.3703    0.1938    0.2545       810
         200     0.6675    0.4390    0.5297       590
         203     0.3696    0.6296    0.4658        27
          22     0.7500    0.7645    0.7572       518
         254     0.1905    0.1176    0.1455        34
         255     0.3333    0.3582    0.3453        67
         264     0.6136    0.4294    0.5053       503
         269     0.1822    0.3868    0.2477       106
         276     0.2373    0.2188    0.2276        64
         284     0.2589    0.2358    0.2468       123
  